In [1]:
import albumentations as A
import cv2
from pathlib import Path
import pandas as pd

# Random OverSampling

In [2]:
labels_df = pd.read_csv('data/fundus/Training_Set/new_RFMiD_Training_Labels.csv')
labels_df['ID'] = labels_df['ID'].astype(str)
class_counts = labels_df.iloc[:, 1:].sum(axis=0)
class_counts

DR       376
ARMD     100
MH       317
DN       138
MYA      101
BRVO      73
TSLN     186
ERM       14
LS        47
MS        15
CSR       37
ODC      282
CRVO      28
TV         6
AH        16
ODP       65
ODE       58
ST         5
AION      17
PT        11
RT        14
RS        43
CRS       32
EDN       15
RPEC      22
MHL       11
RP         6
OTHER     34
dtype: int64

In [3]:
# transform = A.Compose([
#     A.Rotate(limit=45, p=0.5),  # 旋轉，概率為0.5
#     A.HorizontalFlip(p=0.5),    # 水平翻轉，概率為0.5
#     # 添加更多的轉換，如果需要的話
# ])

transform = A.Compose([
    A.OneOf([
        A.Rotate(limit=45, p=1.0),  # 旋转，当被选择时应用的概率为1.0
        A.HorizontalFlip(p=1.0),    # 水平翻转，当被选择时应用的概率为1.0
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0),  # 随机亮度对比度调整
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1.0)  # 色调饱和度调整
    ], p=1)
])

In [4]:
images_path = Path("data/fundus/Training_Set/Training")
ID_name = 1921
while True:
    class_counts = labels_df.iloc[:, 1:].sum(axis=0)
    if all(class_counts >= 100):
        break 
    
    for class_name, count in class_counts.items():
        if count >= 100:
            continue

        images_to_augment = labels_df[labels_df[class_name] == 1]
        img_rows = images_to_augment.sample(n=min(100-count, count))
        # print(img_rows)
        for index, img_row in img_rows.iterrows():
            img_name = img_row['ID']
            image_path = images_path / f"{img_name}.png"
            image = cv2.imread(str(image_path))
            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                transformed = transform(image=image)
                transformed_image = transformed["image"]

                new_img_name = f"{ID_name}.png"
                ID_name += 1
                new_img_path = images_path / new_img_name
                cv2.imwrite(str(new_img_path), cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR))
                
                new_row = img_row.copy()
                new_row['ID'] = new_img_name.replace('.png', '')
                labels_df = labels_df._append(new_row, ignore_index=True)

        break
    # print(class_counts)
    # print("xxxxxxxxxxxxxxxxxxxxxxxx")
            
class_counts

DR       517
ARMD     122
MH       372
DN       203
MYA      103
BRVO     110
TSLN     246
ERM      140
LS       137
MS       129
CSR      138
ODC      440
CRVO     100
TV       100
AH       100
ODP      183
ODE      107
ST       100
AION     100
PT       100
RT       100
RS       101
CRS      110
EDN      101
RPEC     100
MHL      100
RP       104
OTHER    100
dtype: int64

In [5]:
labels_df.to_csv('data/fundus/Training_Set/new_RFMiD_Training_Labels_augmented.csv', index=False)